# Gaussian Discriminant Analysis
This is an implementation of the GDA for image classification. We are going to model the conditional probability distribution p(X|Y=0) and p(X|Y=1) for the negatives and positives samples. Then, using the Bayes rule, we are going to obtain p(Y=0|X) and p(Y=1|X). In fact we are going to obtain a score given that we don't need to calculate p(X) in p(Y|X) = p(X|Y)p(Y)/p(X). X represents the features and p(Y) = 0.5. In this case, the features employed are only related with RGB values and not filters, edges, and related stuff.
 
 **Import packages**
 
**skimage: scikit image for image manipulation**

In [0]:
import skimage
import random
import os
import glob
import numpy as np
from skimage import io



 load files: positives and negatives examples (d = current work directory containing the positives and negatives files)
 

In [0]:
d = os.getcwd()

In [0]:
dp = glob.glob(str(d)+'/positives/*png')
dn= glob.glob(str(d)+'/negatives/*png')

A function that acts like a splitter for cross validation s-fold s = v:

In [0]:
def split(X,v): 
        """generates partitions for the cross validation s-fold (s = v)"""
        pair=[]
        for k in range(len(X)):
		
                pair.append(list((list(X[k]))))
        random.seed(32)
        random.shuffle(pair)
        splen = round(len(pair)/v)
        tt =[]
        for k in range(v):
                uu =[]
                for i in range(k*splen,(k+1)*splen):
                        if i == len(pair):
                                break
                        uu.append(pair[i])
                tt.append(uu)


        return tt, pair


In [0]:
v = 10 # cross validation 10-fold
fp = []

# read images 
for sam in dp: # positive images 
       im = io.imread(sam) # ndarray uint8
       im = np.array(im,dtype='int16')	
       fp.append(im)

fp = np.array(fp)



fn = []
for sam in dn: # negative images  
	im = io.imread(sam)
	im = np.array(im,dtype='int16')	
	fn.append(im)

fn = np.array(fn)

hlistp,pairp= split(fp,v) # sfold huge list of 10 lists each of size 3 ( 30 pos samples / 10-fold = 3)
hlistn,pairn= split(fn,v) # negatives


**In the next steps we are going to perform:**
   - validation and training sets splitting
   - creation of the feature array X containind the 5 color entities. The features are listed as follow:
          Mean(R)
          Mean(G)
          Mean(B)
          Mean(minR,minG,minB)
          Mean(maxR,maxG,maxB)
   - computation of the mean arrays, mu0 and mu1 for the positive and negatives categories respectively as well as the covariance matrix from the training set
   - calculation of the p(X|Y) from the mean arrays and covariance matrix
   \begin{align}
p(X;\mu, \Sigma) & = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}} exp(-\frac{1}{2}(x-\mu)^{T}\Sigma^{-1}(x-\mu))\\
\
\end{align}
   - estimation of the p(Y|X) score for the X in the validation set

In [0]:
for i in range(len(hlistn)):
 	# Here comes the features. Features only based on colors. Mean of R, G, B values and mean of mins and maxs
	X1 =[] # meanR
	X2 =[] # meanG
	X3 =[] # meanB
	X4 =[]# mean (minR minG minB)
	X5 =[] # mean (maxR maxG maxB)
	#The same for negative examples
	X1n =[] # meanR
	X2n =[]
	X3n =[]
	X4n =[]# mean (minR minG minB)
	X5n =[] # mean (maxR maxG maxB)
	l = list(range(10))
	l.remove(i)
	valp = hlistp[i]
	trainp = [hlistp[j] for j in l]
	valn = hlistn[i]	
	trainn = [hlistn[j] for j in l]
	l.insert(i,i)	
	train = trainp + trainn
	val = valp + valn
	train = np.array(train)
	val = np.array(val)
	v = len(val)
	tp = 0 # true positives, true negatives, false posites, and false negatives for the accuracy and confusion matrix
	tn = 0
	fp = 0
	fn = 0
  
	for k in range(len(train)):
		
		Y = train[k]
		for e in range(len(Y)):
			X=Y[e]
			if k < 9: 
				X1.append(np.mean(X[:,:,0]))
				X2.append(np.mean(X[:,:,1]))
				X3.append(np.mean(X[:,:,2]))
				ap = np.mean( [np.min(X[:,:,0]),np.min(X[:,:,1]),np.min(X[:,:,2])] )
				X4.append(ap)
				ap2 = np.mean( [np.max(X[:,:,0]),np.max(X[:,:,1]),np.max(X[:,:,2]) ])
				X5.append(ap2)
				
			else:
				X1n.append(np.mean(X[:,:,0]))
				X2n.append(np.mean(X[:,:,1]))
				X3n.append(np.mean(X[:,:,2]))
				ap = np.mean( [np.min(X[:,:,0]) , np.min(X[:,:,1]) , np.min(X[:,:,2])] )
				X4n.append(ap)
				ap2 = np.mean( [np.max(X[:,:,0]) , np.max(X[:,:,1]) , np.max(X[:,:,2])] )
				X5n.append(ap2)
			

	phi = 0.5 # phi1 = phi2 30 pos and 30 neg
	mu0 = np.array([np.mean(X1n),np.mean(X2n),np.mean(X3n),np.mean(X4n),np.mean(X5n)])#mean array neg
	mu1 = np.array([np.mean(X1),np.mean(X2),np.mean(X3),np.mean(X4),np.mean(X5)])# mean array pos 

	summ=np.zeros(shape=(5,5))
	m =27 
	for ii in range(m):
		a = np.array([X1[ii], X2[ii],X3[ii],X4[ii],X5[ii]])
		b = np.array([X1n[ii], X2n[ii],X3n[ii],X4n[ii],X5n[ii]])
		dif1 = (b - mu0).reshape(-1,1)
		dif1t = np.transpose(dif1)
		mm = np.matmul(dif1,dif1t)
		dif2 = (a - mu1).reshape(-1,1)
		dif2t = np.transpose(dif2)
		mm2 = np.matmul(dif2,dif2t)
		summ = summ + mm + mm2
	

	sig = summ/m*2
	print("partition= ",str(i+1))
	print('mu0= ',mu0,'mu1= ',mu1)
	print('sigma= ',sig)
	siginv=np.linalg.inv(sig)
	det = np.linalg.det(sig)
	### another loop to estimate p(x|y) for validation set
	X1v = []
	X2v = []
	X3v = []
	X4v = []
	X5v = []
	X1vn = []
	X2vn = []
	X3vn = []
	X4vn = []
	X5vn = []
	for k in range(v):
                
		print('Sample ',str(k+1))
		if k < 3:
			X1v.append(np.mean(val[k][:,:,0]))
			X2v.append(np.mean(val[k][:,:,1]))
			X3v.append(np.mean(val[k][:,:,2]))
			X4v.append(np.mean([np.min(val[k][:,:,0]) , np.min(val[k][:,:,1]) , np.min(val[k][:,:,2])]))
			X5v.append(np.mean([np.max(val[k][:,:,0]) , np.max(val[k][:,:,1]) , np.max(val[k][:,:,2])]))
			a = np.array([X1v[k], X2v[k], X3v[k], X4v[k], X5v[k]])
		else:
			X1vn.append(np.mean(val[k][:,:,0]))
			X2vn.append(np.mean(val[k][:,:,1]))
			X3vn.append(np.mean(val[k][:,:,2]))
			X4vn.append(np.mean([np.min(val[k][:,:,0]) , np.min(val[k][:,:,1]) , np.min(val[k][:,:,2])]))
			X5vn.append(np.mean([np.max(val[k][:,:,0]) , np.max(val[k][:,:,1]) , np.max(val[k][:,:,2])]))
			a = np.array([X1vn[k-3], X2vn[k-3], X3vn[k-3], X4vn[k-3], X5vn[k-3]])
		dif1 = (a - mu0).reshape(-1,1)
		dif2 = (a - mu1).reshape(-1,1)
		dif1t = np.transpose(dif1)
		dif2t = np.transpose(dif2)
		p1 = np.matmul(siginv,dif1)
		pp1 = np.dot(dif1t,p1)
		p2 = np.matmul(siginv,dif2)
		pp2 = np.dot(dif1t,p2)
		prob1 = (1/((2*np.pi)**(m/2)*det**(0.5)))*np.exp(-0.5*pp1) # p(x|y=0)
		prob2 = (1/((2*np.pi)**(m/2)*det**(0.5)))*np.exp(-0.5*pp2) # p(x|y=1)

		
		if k < 3:
			print('real=positive')
			
			print('y=0 score = ',prob1*0.5) # we need argmax (p(x|y=0) * p(y)) orginal Bayes: (p(x|y=0) * p(y))/p(x) = p(y=0|x)
			print('y=1 score = ',prob2*0.5) # we need argmax (p(x|y=1) * p(y)) orginal Bayes: (p(x|y=1) * p(y))/p(x) = p(y=1|x)
			if prob2>prob1:
				print('predicted=positive')
				tp += 1
			else:
				print('predicted=negative')
				fn += 1
				
		
		else:
			print('real=negative')

			print('y=0 score = ',prob1*0.5) # we need argmax (p(x|y=0) * p(y)) orginal Bayes: (p(x|y=0) * p(y))/p(x) = p(y=0|x)
			print('y=1 score = ',prob2*0.5) # we need argmax (p(x|y=1) * p(y)) orginal Bayes: (p(x|y=1) * p(y))/p(x) = p(y=1|x)
			if prob1>prob2:
				print('predicted=negative')
				tn += 1
			else:
				print('predicted=positive')
				fp += 1
	print('confusion matrix: ')
	print(str(np.array([[tn,fp],[fn,tp]])))
	print('accuracy= ',str(round((tp+tn)/(tp+fp+tn+fn),2)))

partition=  1
mu0=  [162.58031121 125.11593364 126.15959362 114.97530864 161.80246914] mu1=  [175.51986883 138.87641461 142.84278549  80.54320988 184.07407407]
sigma=  [[680.13317594 740.49301822 539.54626151 533.16902276 452.29190164]
 [740.49301822 963.77351983 647.72771606 710.07659052 508.74903788]
 [539.54626151 647.72771606 501.81836708 484.05959775 419.26209483]
 [533.16902276 710.07659052 484.05959775 695.52385307 336.64228014]
 [452.29190164 508.74903788 419.26209483 336.64228014 496.30605091]]
Sample  1
real=positive
y=0 score =  [[4.58935448e-19]]
y=1 score =  [[8.09226165e-17]]
predicted=positive
Sample  2
real=positive
y=0 score =  [[8.02891029e-21]]
y=1 score =  [[1.17709404e-17]]
predicted=positive
Sample  3
real=positive
y=0 score =  [[4.92496373e-21]]
y=1 score =  [[7.44228772e-18]]
predicted=positive
Sample  4
real=negative
y=0 score =  [[8.58156442e-18]]
y=1 score =  [[8.94393838e-17]]
predicted=positive
Sample  5
real=negative
y=0 score =  [[2.12862943e-17]]
y=1 sco